In [6]:
import pandas as pd 
import numpy as np
import seaborn as sns
import plotly.express as py
import matplotlib.pyplot as plt 

ModuleNotFoundError: No module named 'plotly'

In [7]:
hscore1 = pd.read_csv("2015.csv")
hscore2 = pd.read_csv("2016.csv")
hscore3 = pd.read_csv("2017.csv")
hscore4 = pd.read_csv("2018.csv")
hscore5 = pd.read_csv("2019.csv")

In [ ]:
hscore1.shape

In [ ]:
hscore2.shape

In [ ]:
# check that the shapes look reasonable after combining 
hscore = pd.merge(left=hscore1, right=hscore2, how="inner",on='Country',suffixes = ("_2015", "_2016"))
hscore.shape

In [ ]:
count_missing = hscore.isnull().sum()
print(count_missing.sort_values(ascending = False))

Questions: Which region has the largest number of country surveyed?

In [ ]:
regions = hscore["Region_2015"].value_counts()
fig = py.pie(labels = regions.index, values = regions.values, title = "The percentage of countries in each region", names = regions.index)
fig.show()

In [ ]:
fig = py.pie(labels = regions.index, values = regions.values, title = "The percentage of countries in each region", names = regions.index, hole = 0.7)
fig.show()

Question 2: "which region has is the lowest happiness score? which one is the highest?"

In [ ]:
hscore["happiness_score_combined"] = hscore["Happiness Score_2015"] + hscore["Happiness Score_2016"]
means = hscore.pivot_table('happiness_score_combined', 'Region_2015', aggfunc=np.mean)
means.plot(kind='barh', title='Mean Happiness Scores by Region', xlim=(0,20),legend = False)

In [ ]:
years = [2015,2016,2017,2018,2019]

def clean_col(col):
    col = col.strip()
    col = col.replace("(","")
    col = col.replace(")","")
    col = col.replace(" ","_")
    col = col.replace(".","_")
    col = col.lower()
    return col

df_combined = {}
for a_year in years:
    df = pd.read_csv(f"{a_year}.csv")
    df['Year'] = int(a_year)
    new_columns = []
    for col in df.columns:
        c= clean_col(col)
        new_columns.append(c)
    df.columns = new_columns 
    print(df.columns)
    df_combined[a_year] = df    
    
df_combined[2018].rename({"overall_rank":"happiness_rank"}, axis= 1, inplace=True)  
df_combined[2019].rename({"overall_rank":"happiness_rank"}, axis= 1, inplace=True)   

df_combined[2018].rename({"country_or_region":"country"},axis= 1, inplace=True)
df_combined[2019].rename({"country_or_region":"country"},axis= 1, inplace=True)

df_combined[2018].rename({"score":"happiness_score"},axis= 1, inplace=True)
df_combined[2019].rename({"score":"happiness_score"},axis= 1, inplace=True)

df_combined[2015].rename({"economy_gdp_per_capita":"gdp_per_capita"},axis= 1, inplace=True)
df_combined[2016].rename({"economy_gdp_per_capita_":"gdp_per_capita"},axis= 1, inplace=True)
df_combined[2017].rename({"economy_gdp_per_capita":"gdp_per_capita"},axis= 1, inplace=True)
    
hscore_all = pd.concat(df_combined, ignore_index = True)
hscore_all.sample(10)

In [ ]:
# Before plotting the line graph, one need to convert the string to datetime format
hscore_all['year'] = pd.to_datetime(hscore_all['year'],format='%Y')
hscore_all['year'] = hscore_all['year'].dt.year
print(hscore_all['year'])

Question 3: which countries have the top 5 highest happiness score in year 2019? 

In [ ]:
happiness_2019 = hscore_all[hscore_all['year'] == 2019]

happiness_2019.sort_values(by = "happiness_score")

top_5_2019 = happiness_2019.nlargest(5,"happiness_score")

fig = py.bar(top_5_2019, x="country",y="gdp_per_capita",color ="country")
fig.show()

Comparing three countries on their happiness scores

In [ ]:
countries_of_interest = ['United States', 'Singapore', 'South Korea']
fig = py.bar(hscore_all[hscore_all['country'].isin(countries_of_interest)], 
              x="year", 
              y="happiness_score", 
              color='country', barmode ="group")
fig.show()

Question 4: What is the distribution of happiness score?

In [ ]:
happiness_freq = hscore_all["happiness_score"].value_counts(bins=10).sort_index 
plt.hist(hscore_all["happiness_score"], bins= 20)

In [ ]:
# See what are the countries avaliable in the data set, choose a country of interest 
print(hscore_all["country"].unique())
counts = hscore_all["country"].value_counts().to_dict()
counts["Finland"]

In [ ]:
# Plot a country's happiness score overtime 

In [ ]:
country_of_interest = "China"
china = hscore_all[hscore_all["country"] == "China"]
print(china["happiness_score"])

In [ ]:
plt.figure(figsize=(10,5))
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ylabel('China', fontsize=14)
plt.ylim(0, 10)
plt.title("Change of China's happiness score over time")
plt.plot(china["year"], china["happiness_score"])

In [ ]:
country_of_interest = "Israel"
Israel= hscore_all[hscore_all["country"] == "Iceland"]
print(Israel["happiness_score"])

In [ ]:
plt.figure(figsize=(10,5))
plt.xticks(fontsize=14, rotation=45)
plt.yticks(fontsize=14)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Happiness score', fontsize=14)
plt.ylim(0, 10)
plt.title("Change of happiness score over time")
plt.plot(china["year"], china["happiness_score"],label = "China")
plt.plot(Israel["year"], Israel["happiness_score"],label = "Israel")
plt.legend()
plt.show()

In [ ]:
# What is the relationship between gdp and happiness score?

In [ ]:
hscore_all["happiness_score"].corr(hscore_all["gdp_per_capita"])

In [ ]:
plt.scatter(hscore_all['gdp_per_capita'], hscore_all['happiness_score'])

In [ ]:
plt.scatter(hscore_all['gdp_per_capita'], hscore_all['happiness_score'])
plt.xlabel('gdp_per_capita')
plt.ylabel('happiness_score')
plt.show()

In [ ]:
sns.heatmap(data = hscore_all[['happiness_score', 'gdp_per_capita',
                                     'social_support', 'healthy_life_expectancy',
                                     'freedom_to_make_life_choices', 'generosity',
                                     'perceptions_of_corruption']].corr() , annot = True)